In [50]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.metrics import r2_score, mean_squared_error

from scipy.stats import ttest_1samp, ttest_ind, ttest_rel, shapiro, levene, f_oneway, wilcoxon, mannwhitneyu

from sklearn.linear_model import LinearRegression, LogisticRegression

#### 제3유형. 모집단 1개 (집단의 평균 vs 특정값)

검정방법
1) 정규성 O : 단일표본 검정(ttest_1samp)  (모수 검정)
2) 정규성 X : 윌콕슨 부호순위 검정  (비모수 검정)

가설검정 순서

1) 가설 설정
2) 유의수준 확인
3) 정규성 검정
4) 검정 실시 (통계량, p-value)
5) 귀무가설 기각 여부 결정

In [51]:
df = pd.read_csv('mtcars.csv', encoding='cp949')

df.rename(columns={'Unnamed: 0':'car'}, inplace=True)

df.head(3)

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1


In [55]:
# 예제 문제 1
# mtcars 데이터셋의 mpg열 데이터의 평균이 20과 같다고 할 수 있는지 검정하시오. (유의수준 5%)

# 귀무가설(H0) : 모평균 = 20
# 대립가설(H1) : 모평균 <> 20

# 정규성 검정 (by shapiro)

# 귀무가설(H0) : 정규성을 따른다
# 대립가설(H1) : 정규성을 따르지 않는다

n_s, n_pvalue = shapiro(df['mpg'])
print("검정통계량 : ", n_s)
print("p_value : ", n_pvalue)

if n_pvalue > 0.05 :
  print("정규성을 따른다")
else :
  print("정규성을 따르지 않는다")


t, p_value = ttest_1samp(df['mpg'], popmean=20, alternative='two-sided')

print("검정통계량 : ", t)
print("p_value : ", p_value)

if p_value > 0.05 :
  print("모평균 = 20 (귀무가설 채택)")
else :
  print("모평균 <> 20 (대립가설 채택)")

print("실제 평균 값 : ", round(df['mpg'].mean(),4))

검정통계량 :  0.9475648403167725
p_value :  0.1228824257850647
정규성을 따른다
검정통계량 :  0.08506003568133688
p_value :  0.9327606409093872
모평균 = 20 (귀무가설 채택)
실제 평균 값 :  20.0906


In [53]:
# 만약 정규성을 따르지 않는 경우! 비모수 검정 사용 (윌콕슨 부호순위 검정)
statistic, pvalue = wilcoxon(df['mpg']-20, alternative='two-sided')
print(round(statistic, 4), f"{pvalue:.4f}")

249.0 0.7891


In [59]:
# 예상 문제 2 (단측검정)
# mtcars 데이터셋의 mpg 열 데이터의 평균이 17보다 크다고 할 수 있는지 검정하시오 (유의수준 5%)

# 귀무가설(H0) : 모평균 <= 17
# 대립가설(H1) : 모평균 > 17

n_statistic, n_pvalue = shapiro(df['mpg'])
if n_pvalue > 0.05 :
  print("정규성을 따른다")
else :
  print("정규성을 따르지 않는다")


statistic, pvalue = ttest_1samp(df['mpg'], popmean=17, alternative="greater")

print(round(statistic,4), round(pvalue,4))
if pvalue > 0.05 :
  print("평균이 17보다 작거나 같다")
else :
  print("평균이 17보다 크다")

정규성을 따른다
2.9008 0.0034
평균이 17보다 크다


In [62]:
# 정규성을 만족하지 않는 경우

nx_statistic, nx_pvalue = wilcoxon(df['mpg']-17, alternative='greater')
print(round(nx_statistic,4), round(nx_pvalue,4))
if nx_pvalue > 0.05 :
  print("평균이 17보다 작거나 크다")
else :
  print("평균이 17보다 크다")

395.5 0.0066
평균이 17보다 크다


In [64]:
# 예상 문제 3
# mtcars 데이터셋의 mpg열 데이터의 평균이 17보다 작다고 할 수 있는지 검정하시오 (유의수준 5%)

# 귀무가설(H0) : 평균이 17보다 크거나 같다
# 대립가설(H1) : 평균이 17보다 작다

n_statistic, n_pvalue = shapiro(df['mpg'])
print(round(n_statistic,4), round(n_pvalue,4))

statistic, pvalue = ttest_1samp(df['mpg'], 17, alternative='less')
print(round(statistic,4),round(pvalue,4))

if pvalue > 0.05 :
  print("평균이 17보다 크거나 같다")
else :
  print("평균이 17보다 작다")


0.9476 0.1229
2.9008 0.9966
평균이 17보다 크거나 같다


In [65]:
# 정규성을 만족하지 않을 경우

nx_statistic, nx_pvalue = wilcoxon(df['mpg']-17, alternative='less')

print(round(nx_statistic,4), round(nx_pvalue,4))

395.5 0.9938
